In [13]:
from os import scandir
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from collections import OrderedDict
import pandas as pd
import numpy as np
import tensorflow as tf

DATA = 'samples_csv_back/'
files = [f"{DATA}{file.name}" for file in scandir(DATA) if 'sam' in file.name]

def get_generators(files, val_bird=6, series_len=90):
    print(f'Validation Bird {val_bird}')
    train_files = [file for file in files if int(file[-7]) != val_bird]
    val_files = [file for file in files if file not in train_files]

    x = None

    for file in train_files:
        temp = pd.read_csv(file, header=None)
        if x is None:
            x = temp
        else:
            x = pd.concat([x, temp])

    scaler = StandardScaler()
    scaler.fit(x)
    
    def prepare_data(file):
        x = pd.read_csv(file, header=None)
        x = scaler.transform(x)
        missing = series_len - x.shape[0]
        x = np.pad(x, ((0, missing), (0,0)))
        y = np.array(int(file[-5])-1)
        return x, y
        



    def train_gen():
        for file in train_files:
            x, y = prepare_data(file)
            if x.shape[0] != 90:
                continue
            yield x, y
            
    def val_gen():
        for file in val_files:
            x, y = prepare_data(file)
            if x.shape[0] != 90:
                continue            
            yield x, y
            
    return train_gen, val_gen

def get_labels_predictions(dataset, model):
    preds = []
    labels = []

    for x, y in (dataset):
        preds.append(np.argmax(model.predict(x), axis=1))
        labels.append(y.numpy())
    preds = np.concatenate(preds)
    labels = np.concatenate(labels, axis=0)
    
    return labels, preds

def calculate_summary_stats(labels, preds, name):
    cnf_matrix = confusion_matrix(labels, preds)

    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP) 
    PPV = TP/(TP+FP)
    NPV = TN/(TN+FN)
    FPR = FP/(FP+TN)
    FNR = FN/(TP+FN)
    FDR = FP/(TP+FP)
    
    Recall = TP/(TP+FN)
    
    Informedness = TP/(TP+FN) - FP/(TN+FN)

    ACC = (TP+TN)/(TP+FP+FN+TN)
    
    f1 = 2*((PPV*Recall)/(PPV+Recall))
    
    return {f"{name} Specificity":TNR, f"{name} Accuracy":ACC, f"{name} Recall":Recall,
            f"{name} Precision":PPV, f"{name} Informedness":Informedness, f"{name} F1":f1}

def get_printout_info(model):
    eval_train = model.evaluate(dataset, verbose=0)
    eval_val = model.evaluate(val_dataset, verbose=0)
    
    train_labels, train_preds = get_labels_predictions(dataset, model)
    val_labels, val_preds = get_labels_predictions(val_dataset, model)
    
    train_summary = calculate_summary_stats(train_labels, train_preds, '')
    val_summary = calculate_summary_stats(val_labels, val_preds, 'Val')
    
    return (eval_train, eval_val, train_summary, val_summary)

def format_summary(summary_data):  
    eval_train, eval_val, train_summary, val_summary = summary_data
    
    formatted_data = {"Loss":eval_train[0], "Acc_Total":eval_train[1], 
                      "Val_Loss":eval_val[0], "Val_Acc_Total":eval_val[1]}
    
    for index, current_data in train_summary.items():
        for i in range(current_data.shape[0]):
            formatted_data[f'{index}_{i+1}'] = current_data[i]
            
    for index, current_data in val_summary.items():
        for i in range(current_data.shape[0]):
            formatted_data[f'{index}_{i+1}'] = current_data[i]            
        
    return formatted_data


# In[4]:


def run_trial(dataset, val_dataset, layer, learning_rate):
    model = keras.Sequential([
        keras.layers.InputLayer([90,2]),
        layer,    
        keras.layers.Dense(7, activation="softmax")
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['SparseCategoricalAccuracy'])

    callback = tf.keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)

    history = model.fit(x=dataset, epochs=100000, validation_data=val_dataset, callbacks=[callback])
    
    summary_data = get_printout_info(model)
    trial_summary = format_summary(summary_data)
    
    return trial_summary


learning_rates = [.01]
layers = {"LSTM":keras.layers.LSTM, "GRU":keras.layers.GRU}
activations = ["sigmoid"]

n_trials = 10

for learning_rate in learning_rates:
    
    for name, layer in layers.items():
        for activation in activations:
        
            for units in [20]:

                data_output = {}

                for bird in range(1, 7):

                    for trial in range(1, n_trials+1):

                        train_gen, val_gen = get_generators(files, val_bird=bird)

                        dataset = tf.data.Dataset.from_generator(train_gen, (tf.float32, tf.float32))
                        dataset = dataset.batch(32)

                        val_dataset = tf.data.Dataset.from_generator(val_gen, (tf.float32, tf.float32))
                        val_dataset = val_dataset.batch(32)

                        data_output[f'Val_Bird_{bird}_T{trial}'] = run_trial(dataset, val_dataset, layer(units, activation=activation), learning_rate)

                df = pd.DataFrame.from_dict(data_output, orient='index')
                df.to_csv(f'data/{name}_{units}_{activation}_{learning_rate}.csv')


Validation Bird 1
Epoch 1/100000
6/6 [==============================] - 2s 181ms/step - loss: 1.9804 - sparse_categorical_accuracy: 0.1543 - val_loss: 1.9329 - val_sparse_categorical_accuracy: 0.2000
Epoch 2/100000
6/6 [==============================] - 1s 125ms/step - loss: 1.9267 - sparse_categorical_accuracy: 0.2114 - val_loss: 1.9060 - val_sparse_categorical_accuracy: 0.1714
Epoch 3/100000
6/6 [==============================] - 1s 124ms/step - loss: 1.8972 - sparse_categorical_accuracy: 0.1943 - val_loss: 1.8465 - val_sparse_categorical_accuracy: 0.2571
Epoch 4/100000
6/6 [==============================] - 1s 120ms/step - loss: 1.8432 - sparse_categorical_accuracy: 0.2914 - val_loss: 1.7639 - val_sparse_categorical_accuracy: 0.2286
Epoch 5/100000
6/6 [==============================] - 1s 122ms/step - loss: 1.7573 - sparse_categorical_accuracy: 0.3143 - val_loss: 1.6473 - val_sparse_categorical_accuracy: 0.3429
Epoch 6/100000
6/6 [==============================] - 1s 123ms/step - lo

C:\Users\qazca\AppData\Local\Temp/ipykernel_19284/2130703644.py:126: RuntimeWarning: invalid value encountered in true_divide
  f1 = 2*((PPV*Recall)/(PPV+Recall))


Validation Bird 1
Epoch 1/100000
6/6 [==============================] - 2s 181ms/step - loss: 1.9685 - sparse_categorical_accuracy: 0.1429 - val_loss: 1.9249 - val_sparse_categorical_accuracy: 0.1714
Epoch 2/100000
6/6 [==============================] - 1s 121ms/step - loss: 1.9273 - sparse_categorical_accuracy: 0.2000 - val_loss: 1.9006 - val_sparse_categorical_accuracy: 0.2571
Epoch 3/100000
6/6 [==============================] - 1s 125ms/step - loss: 1.9034 - sparse_categorical_accuracy: 0.2343 - val_loss: 1.8609 - val_sparse_categorical_accuracy: 0.2857
Epoch 4/100000
6/6 [==============================] - 1s 125ms/step - loss: 1.8661 - sparse_categorical_accuracy: 0.2743 - val_loss: 1.8062 - val_sparse_categorical_accuracy: 0.2571
Epoch 5/100000
6/6 [==============================] - 1s 125ms/step - loss: 1.8109 - sparse_categorical_accuracy: 0.3200 - val_loss: 1.7186 - val_sparse_categorical_accuracy: 0.2857
Epoch 6/100000
6/6 [==============================] - 1s 129ms/step - lo

C:\Users\qazca\AppData\Local\Temp/ipykernel_19284/2130703644.py:126: RuntimeWarning: invalid value encountered in true_divide
  f1 = 2*((PPV*Recall)/(PPV+Recall))


Validation Bird 1
Epoch 1/100000
6/6 [==============================] - 3s 180ms/step - loss: 1.9709 - sparse_categorical_accuracy: 0.1943 - val_loss: 1.9295 - val_sparse_categorical_accuracy: 0.2000
Epoch 2/100000
6/6 [==============================] - 1s 125ms/step - loss: 1.9301 - sparse_categorical_accuracy: 0.1771 - val_loss: 1.9120 - val_sparse_categorical_accuracy: 0.1143
Epoch 3/100000
6/6 [==============================] - 1s 126ms/step - loss: 1.9081 - sparse_categorical_accuracy: 0.1829 - val_loss: 1.8680 - val_sparse_categorical_accuracy: 0.2571
Epoch 4/100000
6/6 [==============================] - 1s 124ms/step - loss: 1.8725 - sparse_categorical_accuracy: 0.2629 - val_loss: 1.8122 - val_sparse_categorical_accuracy: 0.2286
Epoch 5/100000
6/6 [==============================] - 1s 137ms/step - loss: 1.8168 - sparse_categorical_accuracy: 0.2514 - val_loss: 1.7260 - val_sparse_categorical_accuracy: 0.2857
Epoch 6/100000
6/6 [==============================] - 1s 135ms/step - lo

...

Validation Bird 5
Epoch 1/100000
6/6 [==============================] - 4s 261ms/step - loss: 1.9896 - sparse_categorical_accuracy: 0.1429 - val_loss: 1.9335 - val_sparse_categorical_accuracy: 0.2000
Epoch 2/100000
6/6 [==============================] - 1s 154ms/step - loss: 1.9380 - sparse_categorical_accuracy: 0.2343 - val_loss: 1.9139 - val_sparse_categorical_accuracy: 0.2286
Epoch 3/100000
6/6 [==============================] - 1s 147ms/step - loss: 1.9098 - sparse_categorical_accuracy: 0.2286 - val_loss: 1.8810 - val_sparse_categorical_accuracy: 0.2571
Epoch 4/100000
6/6 [==============================] - 1s 151ms/step - loss: 1.8765 - sparse_categorical_accuracy: 0.2400 - val_loss: 1.8459 - val_sparse_categorical_accuracy: 0.2571
Epoch 5/100000
6/6 [==============================] - 1s 149ms/step - loss: 1.8339 - sparse_categorical_accuracy: 0.2743 - val_loss: 1.7944 - val_sparse_categorical_accuracy: 0.2857
Epoch 6/100000
6/6 [==============================] - 1s 141ms/step - lo

C:\Users\qazca\AppData\Local\Temp/ipykernel_19284/2130703644.py:126: RuntimeWarning: invalid value encountered in true_divide
  f1 = 2*((PPV*Recall)/(PPV+Recall))


Validation Bird 5
Epoch 1/100000
6/6 [==============================] - 4s 253ms/step - loss: 1.9858 - sparse_categorical_accuracy: 0.1200 - val_loss: 1.9397 - val_sparse_categorical_accuracy: 0.2286
Epoch 2/100000
6/6 [==============================] - 1s 200ms/step - loss: 1.9374 - sparse_categorical_accuracy: 0.2057 - val_loss: 1.9189 - val_sparse_categorical_accuracy: 0.2571
Epoch 3/100000
6/6 [==============================] - 1s 196ms/step - loss: 1.9025 - sparse_categorical_accuracy: 0.2457 - val_loss: 1.8825 - val_sparse_categorical_accuracy: 0.2286
Epoch 4/100000
6/6 [==============================] - 1s 232ms/step - loss: 1.8695 - sparse_categorical_accuracy: 0.2629 - val_loss: 1.8496 - val_sparse_categorical_accuracy: 0.2857
Epoch 5/100000
6/6 [==============================] - 1s 172ms/step - loss: 1.8280 - sparse_categorical_accuracy: 0.3029 - val_loss: 1.8001 - val_sparse_categorical_accuracy: 0.2857
Epoch 6/100000
6/6 [==============================] - 1s 163ms/step - lo

C:\Users\qazca\AppData\Local\Temp/ipykernel_19284/2130703644.py:110: RuntimeWarning: invalid value encountered in true_divide
  PPV = TP/(TP+FP)
C:\Users\qazca\AppData\Local\Temp/ipykernel_19284/2130703644.py:118: RuntimeWarning: invalid value encountered in true_divide
  FDR = FP/(TP+FP)
C:\Users\qazca\AppData\Local\Temp/ipykernel_19284/2130703644.py:126: RuntimeWarning: invalid value encountered in true_divide
  f1 = 2*((PPV*Recall)/(PPV+Recall))


Validation Bird 5
Epoch 1/100000
6/6 [==============================] - 3s 208ms/step - loss: 1.9554 - sparse_categorical_accuracy: 0.1543 - val_loss: 1.9109 - val_sparse_categorical_accuracy: 0.0857
Epoch 2/100000
6/6 [==============================] - 1s 151ms/step - loss: 1.9092 - sparse_categorical_accuracy: 0.1886 - val_loss: 1.8679 - val_sparse_categorical_accuracy: 0.2286
Epoch 3/100000
6/6 [==============================] - 1s 146ms/step - loss: 1.8685 - sparse_categorical_accuracy: 0.2514 - val_loss: 1.8172 - val_sparse_categorical_accuracy: 0.3143
Epoch 4/100000
6/6 [==============================] - 1s 145ms/step - loss: 1.8209 - sparse_categorical_accuracy: 0.2457 - val_loss: 1.7606 - val_sparse_categorical_accuracy: 0.2857
Epoch 5/100000
6/6 [==============================] - 1s 169ms/step - loss: 1.7583 - sparse_categorical_accuracy: 0.2914 - val_loss: 1.6771 - val_sparse_categorical_accuracy: 0.3429
Epoch 6/100000
6/6 [==============================] - 1s 177ms/step - lo